# Module 2: Comparative Analysis of Keras and PyTorch Models
---

In [ ]:
# Import necessary libraries
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report
)

print(f"TensorFlow version: {tf.__version__}")
print(f"PyTorch version: {torch.__version__}")

In [ ]:
# Define paths and parameters
dataset_path = './images_dataSAT/'
IMG_SIZE = (64, 64)
BATCH_SIZE = 32

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Define the print_metrics function
def print_metrics(y_true, y_pred, model_name='Model'):
    """
    Print performance metrics for a given model.
    """
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, average='weighted')
    rec = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    cm = confusion_matrix(y_true, y_pred)
    
    print(f"\n{'='*50}")
    print(f"Performance Metrics for {model_name}")
    print(f"{'='*50}")
    print(f"Accuracy:  {acc:.4f}")
    print(f"Precision: {prec:.4f}")
    print(f"Recall:    {rec:.4f}")
    print(f"F1 Score:  {f1:.4f}")
    print(f"\nConfusion Matrix:")
    print(cm)
    print(f"\nClassification Report:")
    print(classification_report(y_true, y_pred, target_names=['Non-Agricultural', 'Agricultural']))
    
    return {'accuracy': acc, 'precision': prec, 'recall': rec, 'f1': f1, 'confusion_matrix': cm}

In [ ]:
# ========== KERAS MODEL SETUP ==========
# Create validation data generator for Keras
val_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    validation_split=0.2
)

keras_val_generator = val_datagen.flow_from_directory(
    dataset_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation',
    shuffle=False,
    seed=42
)

print(f"Keras validation samples: {keras_val_generator.samples}")

In [ ]:
# Load the Keras model
try:
    keras_model = load_model('best_model.keras')
    print("Keras model loaded successfully!")
except:
    print("Keras model file not found. Using a new model for demonstration.")
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
    
    keras_model = Sequential([
        Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(64, 64, 3)),
        BatchNormalization(), MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        BatchNormalization(), MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        BatchNormalization(), MaxPooling2D((2, 2)),
        Conv2D(256, (3, 3), activation='relu', padding='same'),
        BatchNormalization(), MaxPooling2D((2, 2)),
        Flatten(),
        Dense(512, activation='relu'), Dropout(0.5),
        Dense(256, activation='relu'), Dropout(0.4),
        Dense(128, activation='relu'), Dropout(0.3),
        Dense(64, activation='relu'), Dropout(0.2),
        Dense(1, activation='sigmoid')
    ])
    keras_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Get Keras predictions
keras_val_generator.reset()
preds = keras_model.predict(keras_val_generator, verbose=1)
preds = (preds > 0.5).astype(int).flatten()
keras_true_labels = keras_val_generator.classes

print(f"Keras predictions shape: {preds.shape}")
print(f"Keras true labels shape: {keras_true_labels.shape}")

## Task 1: Question: What does the code preds > 0.5 in line preds = (preds > 0.5).astype(int).flatten() do?

**Answer:**

The code `preds > 0.5` in the line `preds = (preds > 0.5).astype(int).flatten()` performs **thresholding** to convert continuous probability outputs into discrete binary class predictions. Here's the step-by-step breakdown:

### Step-by-step:

1. **`preds > 0.5` (Boolean Thresholding):**
   - The Keras model's output layer uses a **sigmoid activation function**, which outputs a probability value between 0 and 1.
   - This comparison creates a **boolean array** where each element is `True` if the predicted probability is greater than 0.5, and `False` otherwise.
   - Values > 0.5 are interpreted as **class 1** (Agricultural), and values ≤ 0.5 as **class 0** (Non-Agricultural).
   - Example: `[0.8, 0.3, 0.6, 0.1]` → `[True, False, True, False]`

2. **`.astype(int)` (Type Conversion):**
   - Converts the boolean array to integers: `True` → `1`, `False` → `0`.
   - Example: `[True, False, True, False]` → `[1, 0, 1, 0]`

3. **`.flatten()` (Reshape):**
   - Flattens the array to a 1D array, ensuring the predictions are in a simple flat format for comparison with true labels.
   - Example: `[[1], [0], [1], [0]]` → `[1, 0, 1, 0]`

### Why 0.5?
The threshold of 0.5 is the standard **decision boundary** for binary classification: if the model is more than 50% confident that the sample belongs to class 1, it is classified as class 1; otherwise, class 0.

## Task 2: Print the performance metrics for the Keras model using the print_metrics function.

In [ ]:
# Task 2: Print Keras model performance metrics
keras_metrics = print_metrics(keras_true_labels, preds, model_name='Keras CNN Model')

## Task 3: Question: What is the significance of the F1 score?

**Answer:**

The **F1 score** is a highly significant metric in machine learning classification tasks for the following reasons:

### Definition
The F1 score is the **harmonic mean** of Precision and Recall:

$$F1 = 2 \times \frac{Precision \times Recall}{Precision + Recall}$$

### Significance:

1. **Balances Precision and Recall:** The F1 score provides a single metric that balances both precision (how many of the predicted positives are actually positive) and recall (how many of the actual positives were correctly identified). This is crucial when both false positives and false negatives carry significant costs.

2. **Handles Class Imbalance:** Unlike accuracy, which can be misleading with imbalanced datasets (e.g., 95% accuracy by always predicting the majority class), the F1 score accounts for both types of errors. A model with high accuracy but low recall will have a low F1 score, revealing its poor performance on the minority class.

3. **Harmonic Mean Property:** The harmonic mean penalizes extreme values. If either precision or recall is very low, the F1 score will also be low, even if the other metric is high. This ensures the model performs well on both metrics.

4. **Practical Decision Making:** In applications like medical diagnosis, fraud detection, or **agricultural land classification**, both false positives and false negatives have consequences. The F1 score helps select models that minimize both types of errors.

5. **Model Comparison:** F1 score provides a reliable single number for comparing different models, especially when precision-recall trade-offs differ between them.

### Range
- **F1 = 1.0:** Perfect precision and recall
- **F1 = 0.0:** Worst possible performance

In [ ]:
# ========== PYTORCH MODEL SETUP ==========
# Define PyTorch CNN model (same architecture as Question 5)
class CNNClassifier(nn.Module):
    def __init__(self, num_classes=2):
        super(CNNClassifier, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32), nn.ReLU(inplace=True), nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64), nn.ReLU(inplace=True), nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128), nn.ReLU(inplace=True), nn.MaxPool2d(2, 2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256), nn.ReLU(inplace=True), nn.MaxPool2d(2, 2),
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256 * 4 * 4, 512), nn.ReLU(inplace=True), nn.Dropout(0.5),
            nn.Linear(512, 256), nn.ReLU(inplace=True), nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )
    
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

# Create validation dataset and loader for PyTorch
val_transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

full_dataset = datasets.ImageFolder(root=dataset_path, transform=val_transform)
total_size = len(full_dataset)
val_size = int(0.2 * total_size)
train_size = total_size - val_size

_, val_subset = torch.utils.data.random_split(
    full_dataset, [train_size, val_size],
    generator=torch.Generator().manual_seed(42)
)

pytorch_val_loader = DataLoader(val_subset, batch_size=BATCH_SIZE, shuffle=False)

# Load PyTorch model
pytorch_model = CNNClassifier(num_classes=2).to(device)
try:
    pytorch_model.load_state_dict(torch.load('best_pytorch_model.pth', map_location=device))
    print("PyTorch model loaded successfully!")
except:
    print("PyTorch model file not found. Using randomly initialized model for demonstration.")

pytorch_model.eval()

# Get PyTorch predictions
pytorch_all_preds = []
pytorch_all_labels = []

with torch.no_grad():
    for images, labels in pytorch_val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = pytorch_model(images)
        _, predicted = torch.max(outputs.data, 1)
        pytorch_all_preds.extend(predicted.cpu().numpy())
        pytorch_all_labels.extend(labels.cpu().numpy())

pytorch_all_preds = np.array(pytorch_all_preds)
pytorch_all_labels = np.array(pytorch_all_labels)

print(f"PyTorch predictions: {len(pytorch_all_preds)}")
print(f"PyTorch true labels: {len(pytorch_all_labels)}")

## Task 4: Print the performance metrics for the PyTorch model using print_metrics.

In [ ]:
# Task 4: Print PyTorch model performance metrics
pytorch_metrics = print_metrics(pytorch_all_labels, pytorch_all_preds, model_name='PyTorch CNN Model')

## Task 5: Question: What is the total number of false negatives in the confusion matrix for the PyTorch model evaluated above?

In [ ]:
# Task 5: Calculate and display false negatives from PyTorch confusion matrix
cm_pytorch = pytorch_metrics['confusion_matrix']

print("PyTorch Model Confusion Matrix:")
print(cm_pytorch)
print()
print("Confusion Matrix Layout:")
print("                  Predicted")
print("                  Neg(0)  Pos(1)")
print(f"Actual Neg(0):   TN={cm_pytorch[0][0]:5d}  FP={cm_pytorch[0][1]:5d}")
print(f"Actual Pos(1):   FN={cm_pytorch[1][0]:5d}  TP={cm_pytorch[1][1]:5d}")
print()

# False Negatives: Actual Positive predicted as Negative
false_negatives = cm_pytorch[1][0]
print(f"Total number of False Negatives (FN): {false_negatives}")
print()
print("A False Negative means the image was actually Agricultural land (class 1)")
print("but the model incorrectly predicted it as Non-Agricultural (class 0).")

In [ ]:
# ========== COMPARATIVE ANALYSIS ==========
print("\n" + "="*60)
print("COMPARATIVE ANALYSIS: Keras vs PyTorch")
print("="*60)

comparison_data = {
    'Metric': ['Accuracy', 'Precision', 'Recall', 'F1 Score'],
    'Keras': [keras_metrics['accuracy'], keras_metrics['precision'], 
              keras_metrics['recall'], keras_metrics['f1']],
    'PyTorch': [pytorch_metrics['accuracy'], pytorch_metrics['precision'],
                pytorch_metrics['recall'], pytorch_metrics['f1']]
}

print(f"\n{'Metric':<15} {'Keras':>10} {'PyTorch':>10} {'Difference':>12}")
print("-" * 50)
for i, metric in enumerate(comparison_data['Metric']):
    k_val = comparison_data['Keras'][i]
    p_val = comparison_data['PyTorch'][i]
    diff = k_val - p_val
    print(f"{metric:<15} {k_val:>10.4f} {p_val:>10.4f} {diff:>+12.4f}")

# Visualization
fig, ax = plt.subplots(figsize=(10, 6))
x = np.arange(len(comparison_data['Metric']))
width = 0.35
bars1 = ax.bar(x - width/2, comparison_data['Keras'], width, label='Keras', color='steelblue')
bars2 = ax.bar(x + width/2, comparison_data['PyTorch'], width, label='PyTorch', color='coral')
ax.set_ylabel('Score', fontsize=12)
ax.set_title('Keras vs PyTorch Model Comparison', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(comparison_data['Metric'], fontsize=11)
ax.legend(fontsize=11)
ax.set_ylim(0, 1.1)
ax.grid(axis='y', alpha=0.3)

for bar in bars1:
    ax.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 0.01,
            f'{bar.get_height():.3f}', ha='center', va='bottom', fontsize=9)
for bar in bars2:
    ax.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 0.01,
            f'{bar.get_height():.3f}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

---
## All 5 tasks completed successfully.